
### Explicació script Bronze to Silver

**Quin objectiu té la capa Silver en aquest cas?**

- L'objectiu d'aquesta capa és transformar les dades brutes en dades netes. Amb "netes" ens referim a dades sense columnes innecessàries, en el format adequat, organitzades correctament i amb camps afegits si cal.
- Això ens permetrà realitzar moltes accions sobre aquesta capa, ja que tindrem les dades netejades i preparades per a l'anàlisi.

**Quins processos i accions realitzem en aquest script? (EXPLICACIÓ PAS A PAS)**

- Agafem les dades en format JSON de la capa Bronze i les convertim a dataframes de pandas per poder-les tractar.
- Creem diferents funcions per poder extreure les dades que ens interessen de la capa Bronze i per mostrar-ne el funcionament.
- Una vegada fetes aquestes funcions, llegim totes les dades de la capa Bronze. Durant aquest pas, afegim dades del tiquet com l'ID, que ens interessen i que extraiem dels directoris en aquest cas.
- Quan tenim totes les dades en format pandas, les tractem, és a dir:
  - Netegem les dades innecessàries dels camps que tenim.
  - Eliminem columnes innecessàries a partir d'una funció creada.
  - Convertim les dades al format adequat.
  - Afegim camps que falten.
- Creem dues funcions per particionar les dades per clients o per dies en format Parquet.
- Un cop definides les funcions, emmagatzemem les dades en dos directoris diferents a la capa Silver del Data Lake, particionades de diferents maneres en cada capa.

#### Importem les llibreries necessàries

In [0]:
import numpy as np
import pandas as pd


#### Configuració connexió Spark
- En aquest codi podem veure les variables necessàries per realitzar la connexió amb Spark i com creem l'objecte Spark amb aquestes variables.
- La connexió en aquest cas la fem amb SAS, ja que només ens fa falta un token per poder-la establir. Hi ha més maneres de realitzar la connexió.

In [0]:
# Configuració Spark per connexió ADLG2 amb token SAS
token = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-03T02:34:04Z&st=2024-04-09T18:34:04Z&spr=https&sig=dplYYYNSRC%2FjDR89WMxcsA6SMo%2BHFYrQ5BqdhRUKBMs%3D'
storage_account = 'projecteiabd'
container = 'bronze'

spark.conf.set("fs.azure.account.auth.type.{0}.dfs.core.windows.net".format(storage_account), "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.{0}.dfs.core.windows.net".format(storage_account), "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.{0}.dfs.core.windows.net".format(storage_account), token)

#### Lectura d'un document amb Spark
- Ara veurem un exemple de com llegir un document concret amb Spark.
- Utilitzem la ruta absoluta del document, que és la del nostre Azure Data Lake.
- Indiquem que és multiLine perquè si no, no llegeix bé el JSON, ja que tenim més d'un objecte dins del JSON.

In [0]:
# Exemple de com llegir un document del datalake amb json.
# path = 'abfs://{0}@{1}.dfs.core.windows.net/1/21052024/21052024_20051716314645.json'.format(container, storage_account)

# df = spark.read.json(path, multiLine=True)

# display(df)

#### Lectura de múltiples documents (JSON)

- A continuació, veurem un exemple de com llistar els fitxers i carpetes que es troben dins una carpeta del Data Lake d'Azure.
- En aquest exemple, observem que hi ha dues carpetes.
- A més, podem veure detalls com la ruta completa, el nom del fitxer/camí i la data de modificació.

In [0]:
# Exemple de com llistar fitxers i carpetas que hi ha dintre de un path.
path = 'abfs://{0}@{1}.dfs.core.windows.net/'.format(container, storage_account)

files_in_adls = dbutils.fs.ls(path)
print(files_in_adls)

[FileInfo(path='abfs://bronze@projecteiabd.dfs.core.windows.net/1/', name='1/', size=0, modificationTime=1716568861000), FileInfo(path='abfs://bronze@projecteiabd.dfs.core.windows.net/2/', name='2/', size=0, modificationTime=1716568873000), FileInfo(path='abfs://bronze@projecteiabd.dfs.core.windows.net/3/', name='3/', size=0, modificationTime=1716568922000), FileInfo(path='abfs://bronze@projecteiabd.dfs.core.windows.net/4/', name='4/', size=0, modificationTime=1716568890000)]



#### Declaració de funcions.

A continuació, declararem les funcions que utilitzarem per tractar i penjar les dades a la capa Silver.

- **Funció per llistar tots els documents que hi ha dins d'una ruta**
  - Aquesta funció permet llistar tots els fitxers que hi ha dins una ruta; en aquest cas, s'utilitzarà per revisar tots els fitxers que conté un container.
  - La funció és especialment útil perquè pot llegir fitxers dins de subcarpetes de manera recursiva fins a obtenir tots els fitxers.
  - Retorna una llista d'objectes que representen els fitxers, incloent-hi dades com el path, name, i modificationTime.

- **Funció per eliminar columnes innecessàries**
  - Aquesta funció permet eliminar columnes no necessàries d'un DataFrame de manera ràpida i senzilla.
  - Només cal passar una llista de les columnes que no es volen i el DataFrame del qual s'han d'eliminar.
  - Retorna el DataFrame amb les columnes eliminades.

- **Funció per visualitzar els productes d'un DataFrame**
  - Aquesta funció serveix per analitzar les dades, mostrant com estan distribuïts els productes.
  - Realitza prints que permeten visualitzar les dades

- **Funció per emmagatzemar les dades particionades per clients**
  - Aquesta funció permet emmagatzemar les dades del DataFrame final en un repositori remot, en aquest cas un container de Azure Data Lake.
  - Les dades es particionen per id_client.

- **Funció per emmagatzemar les dades particionades per dies**
  - Aquesta funció permet emmagatzemar les dades del DataFrame final en un repositori remot, en aquest cas un container de Azure Data Lake, particionant per dies.

- **Funció per canviar les "," per "." a l'import, preu_unitari i total**
  - Aquesta funció canvia les "," per "." a les columnes necessàries (import, preu_unitari, total).

- **Funció per eliminar kg de la columne preu_unitat**
  - Aquesta funció elimina el text "kg" de la columna preu_unitari.

- **Funció per crear una nova columna unitats**
  - En aquesta funció determinem sí és unitat kg o ud el producte.
  - Això ens serveix per extreure el kg de la quanitat. Ja que això pot donar errors.


In [0]:
# Funció per llistar tots els documents que hi ha dins de una ruta.

def deep_ls(path): # Definim una funció que li passem un path.
    """List all files in base path recursively."""
    for x in dbutils.fs.ls(path): # Agafem tots els fitxer i carpetas que hi ha dintre el path. I els anem recorrent. 
        if x.path[-1] != '/': # Agefm l'ultim caracter de la ruta del fitxer, mirem que si no és un '/' vol dir que és un fitxer.
            yield x # Retornem el fitxer amb un yield osigui que retorna el fitxer però segueix amb l'execuió fins que no quedin més docs.
        else: 
            for y in deep_ls(x.path): # Si no és un fitxer entrem dintre la carpeta, amb la mateixa funció de aqui bé la recursivitat. Que no parara de executar-se fins que ho hagi recorregut tot.
                yield y # Anem retornan els resultats de cada fitxer que va trobant la funció.

# FUNCIÓ YIELD
# No retorna tots els valors de una tirada, sino que els va retornant a mesura que va executa-nse.
# Es com un generador, anirà donant tots els fitxers fins que ja no en tigui més per donar.
# Per això fem un list(def) per emmegatzemar totes les sortides del yield en una llista. 

# Exemple funcionament: 
files = list(deep_ls(path)) # Emmegatzamem tots els resultats de la funció en una llista. Bàsicament acumulem tots els resultats.
for file in files: 
    print(file.path) # Mostrem tots els documents que hem trobat.

abfs://bronze@projecteiabd.dfs.core.windows.net/1/24052024/24052024_184100.json
abfs://bronze@projecteiabd.dfs.core.windows.net/2/24052024/24052024_184112.json
abfs://bronze@projecteiabd.dfs.core.windows.net/2/24052024/24052024_184138.json
abfs://bronze@projecteiabd.dfs.core.windows.net/2/24052024/24052024_184507.json
abfs://bronze@projecteiabd.dfs.core.windows.net/3/24052024/24052024_184201.json
abfs://bronze@projecteiabd.dfs.core.windows.net/3/24052024/24052024_184226.json
abfs://bronze@projecteiabd.dfs.core.windows.net/4/24052024/24052024_184128.json
abfs://bronze@projecteiabd.dfs.core.windows.net/4/24052024/24052024_184542.json


In [0]:
# Funció per eliminar columnes inecessaries.
def deleteColumns(df, columns_list):

  df_docs.drop(columns_list, inplace=True, axis = 1)

  return df_docs

In [0]:
# Funció per visualitzar els productes de un df.
def visualitzarProducts(df_products):
    for index, llista_productes in df_products['products'].items():
        print("Tiquet ", index + 1)
        print()
        print("--Productes--")
        for producte in llista_productes:
            print("Nom:", producte['descripcio'])
            print("Qty:", producte['quantitat'])
            print("Import:", producte['import'], "€")
            print('preu_unitari:', producte['preu_unitari'], '€')
            print('unitat:', producte['unitat'])
            print('')

In [0]:
# Funció per emmegatzemar en format parquet les dades a un repositori extern. Particionades per id_client.
def particionar_clients(df, path):
    # Passem el df de pandas a spark df.
    df = spark.createDataFrame(df)

    df.write.format("parquet") \
        .option("forward_spark_azure_storage_credentials", "True") \
        .partitionBy("id_client") \
        .mode("overwrite") \
        .save(path)

In [0]:
# Funció per emmegatzemar en format parquet les dades a un repositori extern. Particionades per dies.
def particionar_dies(df, path):
    # Passem el df de pandas a spark df.
    df = spark.createDataFrame(df)

    df.write.format("parquet") \
        .option("forward_spark_azure_storage_credentials", "True") \
        .partitionBy("data") \
        .mode("overwrite") \
        .save(path)

In [0]:
# Funció per eliminar el text "kg" de la columna preu unitats.
def eliminar_txt_kg(df):
    for index, llista_productes in df['products'].items():
        for producte in llista_productes:
            if 'preu_unitari' in producte:
                # Elimina el text " €/kg €"
                producte['preu_unitari'] = re.sub(r' ?€/kg ?', '', producte['preu_unitari'])
    return df


In [0]:
# Funció per canviar les , per . a l'import i el preu_unitari
def canviar_punts_per_comes(row):
    row['total_amb_IVA'] = str(row['total_amb_IVA']).replace(',', '.')
    for producte in row['products']:
        producte['import'] = str(producte['import']).replace(',', '.')
        if 'preu_unitari' in producte:
            producte['preu_unitari'] = str(producte['preu_unitari']).replace(',', '.')
    return row

In [0]:
# Funció per crear una nova columna 'unitat' i treure el text "kg" de 'preu_unitari'.
def crear_columna_unitats(df):
    for index, llista_productes in df['products'].items():
        for producte in llista_productes:
            if 'kg' in producte['preu_unitari']:
                producte['quantitat'] = re.sub(r' ?kg ?', '', producte['quantitat'])
                producte['unitat'] = 'kg'
            else:
                producte['unitat'] = 'ud'
    return df

#### Llegim les dades del Data Lake d'Azure
- En aquest apartat comencem el procés de llegir les dades. Ho fem amb la funció que hem desenvolupat anteriorment per llistar tots els fitxers dins d'una ruta.
- També afegim columnes addicionals a les dades que llegim: id_client, id_tiquet i data_scanner_tiquet.
  - Aquestes columnes ens serveixen per poder classificar les dades que obtenim per client i tiquet.
  - Tenen un gran valor informatiu, ja que ens permeten desenvolupar i analitzar les dades de manera més útil i eficient.
- Finalment, retornem un DataFrame de Pandas amb totes les dades dels fitxers combinades.

In [0]:
# Utilitzem la funció anterior, per llegir tots els json, i posar-lo a un DF.
docs = []

files = list(deep_ls(path)) 

# Recorrem la llista de fitxers.
for file in files: 
    # Llegim els fitxers.
    df = spark.read.json(file.path, multiLine=True)
    
    # Guardem la futa del fitxer
    ruta = str(file.path)

    # Passem el df a df de pandas
    df = df.toPandas()

    # Afegim al df, els camps id_client, id_tiquet, data_tiquet. Ja que aquets camps ens serviarn per classificar els tiquets.
    df['id_tiquet'] = ruta.split("/")[-1].split(".")[0]; # Agragem l'id del tiquet al df que serà el nom del fitxer.
    df['id_client'] = ruta.split("/")[-3] # Agafem l'id del client.
    df['data_scanner_tiquet'] = ruta.split("/")[-2] # Agafem la data.

    # Anem afegint els df a la llista.
    docs.append(df)

# Fem un bucle dintre un array per posar tots els df dintre. Els df que hi ha a la llista docs.
df_docs = pd.concat([df for df in docs], ignore_index=True)

df_docs

,AID,ARC,AUT,NC,adressa,barcode,ciutat,data,factura_simplificada,iva_total,...,products,targeta_1,targeta_2,targeta_bancaria,telf,total_amb_IVA,total_sense_IVA,id_tiquet,id_client,data_scanner_tiquet
0,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,VILADECANS 08840,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'LASANYA', 'import': '17,60', ...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"17,60","AAA,AA",24052024_184100,1,24052024
1,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,REUS 43205,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'ARROS NEGRE', 'import': '2.3)...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"30,8]","AAA,AA",24052024_184112,2,24052024
2,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,BARCELONA 08025,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'ANELLES POTA', 'import': '4.3...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"42,00","AAA,AA",24052024_184138,2,24052024
3,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,BARCELONA 08038,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'CROQUETES', 'import': '1,05',...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"1,38","AAA,AA",24052024_184507,2,24052024
4,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,SANTA COLOMA DE FARNERS 17430,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'IOGURT NATURAL', 'import': '3...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"58,90","AAA,AA",24052024_184201,3,24052024
5,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,BARCELONA 08019,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'CALAMARSO.', 'import': '16,50...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"19,83","AAA,AA",24052024_184226,3,24052024
6,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,VILANOVA I LA GELTRÚ 08800,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'CANELONS', 'import': '4.62', ...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"23,95","AAA,AA",24052024_184128,4,24052024
7,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,MONTORNÉS DEL VALLES 08170,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'PATATES GRILL', 'import': '1,...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"3,50","AAA,AA",24052024_184542,4,24052024


#### Analitzem les dades.
En aquest apartat, analitzarem les dades per determinar quines transformacions cal realitzar abans de passar-les a la capa Silver. Aquest anàlisi es realitza aquí, però en l'entorn de producció no és necessari, ja que optimitzem el codi i aquests anàlisis ja s'hauran fet anteriorment.

- Analitzem els tipus de dades que tenen les columnes:
    - Ens assegurem que els tipus de dades són els correctes per a cada columna (ex: numèric, text, data).
    - Convertim els tipus de dades quan sigui necessari per facilitar el processament posterior.

- Verifiquem que el camp id_tiquet no estigui repetit:
    - Ens assegurem que cada tiquet té un identificador únic per evitar duplicats i inconsistències.

- Comprovem si hi ha valors nuls o buits:
    - Identifiquem i gestionem els valors nuls o buits per evitar errors en els passos de processament futurs.
    - Podem optar per eliminar registres incomplets o omplir els valors nuls amb valors per defecte.

- Analitzem els productes de cada tiquet i les seves dades:
    - Revisem la informació dels productes associats a cada tiquet per assegurar-nos que totes les dades necessàries estan presents i correctes.

- Analitzem els valors i que siguin els correctes:
    - Validem que els valors dels camps siguin coherents i correctes segons les especificacions del projecte (ex: formats de preus, quantitats, dates).

In [0]:
# Analitzem els tipus de les columnes.
df_docs.dtypes

Out[217]: AID                       object
ARC                       object
AUT                       object
NC                        object
adressa                   object
barcode                   object
ciutat                    object
data                      object
factura_simplificada      object
iva_total                 object
ivas                      object
logo_bottom               object
logo_contact_less         object
logo_top                  object
nom                       object
op                        object
pagat_targeta_bancaria    object
products                  object
targeta_1                 object
targeta_2                 object
targeta_bancaria          object
telf                      object
total_amb_IVA             object
total_sense_IVA           object
id_tiquet                 object
id_client                 object
data_scanner_tiquet       object
dtype: object

In [0]:
# Ens assegurem que a la columna 'id_tiquet' no es repeteixin.

# Amb .ducpliacated de pandas, podem agafar els valors duplicats. En aquest cas indiquem del camp id_tiquet
duplicats = df_docs.duplicated(subset=['id_tiquet'])

# Sumem el número de resultats.
num_duplicats = duplicats.sum()

# Si hi ha duplicats ho indiquem, i els mostrem amb un print.
# En cas que no ho indiquem.
if num_duplicats > 0:
    print(f"Hi ha {num_duplicats} id_tiquet duplicats.")
    print("Els ID duplicats són:")
    print(df_docs[duplicats]['id_tiquet'])
    # Eliminem els duplicats i només mentenim la primera aparició
    df_docs = df_docs.drop_duplicates(subset=['id_tiquet'])
else:
    print("No hi ha cap id_tiquet duplicat.")


No hi ha cap id_tiquet duplicat.


In [0]:
# Comprovem si hi ha valors nuls o buits a les columnes
valors_nuls = df_docs.isnull().sum()

if valors_nuls.sum() > 0:
    print("Hi ha valors nuls o buits a les següents columnes:")
    print(valors_nuls[valors_nuls > 0])
else:
    print("No hi ha valors nuls o buits en cap columna.")

No hi ha valors nuls o buits en cap columna.


In [0]:
# Analitzem els valors i que siguin els correctes.
df_docs.head(5)

,AID,ARC,AUT,NC,adressa,barcode,ciutat,data,factura_simplificada,iva_total,...,products,targeta_1,targeta_2,targeta_bancaria,telf,total_amb_IVA,total_sense_IVA,id_tiquet,id_client,data_scanner_tiquet
0,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,VILADECANS 08840,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'LASANYA', 'import': '17,60', ...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"17,60","AAA,AA",24052024_184100,1,24052024
1,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,REUS 43205,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'ARROS NEGRE', 'import': '2.3)...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"30,8]","AAA,AA",24052024_184112,2,24052024
2,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,BARCELONA 08025,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'ANELLES POTA', 'import': '4.3...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"42,00","AAA,AA",24052024_184138,2,24052024
3,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,BARCELONA 08038,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'CROQUETES', 'import': '1,05',...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"1,38","AAA,AA",24052024_184507,2,24052024
4,B0000000012345,1234,123456,123456789,"C/ JOSEP MARÍA FOLCH I TORRES, 5",image,SANTA COLOMA DE FARNERS 17430,22-04-2024,1234-123-12345,"50,33",...,"[{'descripcio': 'IOGURT NATURAL', 'import': '3...",VISA CREDITO/DEB,Visa CaixaBank,**** **** **** **** 1234,977598991,"58,90","AAA,AA",24052024_184201,3,24052024



#### Tractament de dades
Un cop hem analitzat les dades, realitzem els tractaments necessaris per assegurar-nos que les dades estan netes i estructurades correctament. Aquests són els passos que seguim:

- Eliminació de columnes innecessàries:
    - Identifiquem i eliminem les columnes que no aporten valor al nostre anàlisi.

- Neteja de camps:
    - Per exemple, al camp ciutat, eliminem el codi postal si està present.

- Conversió de dates:
    - Assegurem que totes les dates estan en el format de dades corresponent per facilitar el seu ús posterior.

- Afegim una nova columne "unitat" i eliminem el text "kg" de la columne "quanitat"
    - D'aquesta forma desglossem la unitat de la quantitat

- Canvi de separadors decimals:
    - Canviem els punts (.) per comes (,) en els imports i preus unitaris dels productes i al total del tiquet, ja que el model OCR pot detectar els dos formats i necessitem unificar-los.

- Eliminació del text "kg" a la columna preu_unitat:
    - El text "kg" no és necessari per al nostre anàlisi, així que el retirem.

In [0]:
# Fem ús de la funció deleteColumns() per eliminar les columnes inecessaries.
columns_to_drop = ['ivas', 'targeta_bancaria', 'NC', 'AUT', 'ARC', 'AID', 'telf', 'nom', 'total_sense_IVA', 'adressa', 'targeta_1', 'targeta_2', 'targeta_bancaria', 'logo_top', 'op', 'logo_contact_less', 'logo_bottom', 'iva_total', 'barcode', 'pagat_targeta_bancaria']

df_docs = deleteColumns(df_docs, columns_to_drop)

df_docs

,ciutat,data,factura_simplificada,products,total_amb_IVA,id_tiquet,id_client,data_scanner_tiquet
0,VILADECANS 08840,22-04-2024,1234-123-12345,"[{'descripcio': 'LASANYA', 'import': '17,60', ...","17,60",24052024_184100,1,24052024
1,REUS 43205,22-04-2024,1234-123-12345,"[{'descripcio': 'ARROS NEGRE', 'import': '2.3)...","30,8]",24052024_184112,2,24052024
2,BARCELONA 08025,22-04-2024,1234-123-12345,"[{'descripcio': 'ANELLES POTA', 'import': '4.3...","42,00",24052024_184138,2,24052024
3,BARCELONA 08038,22-04-2024,1234-123-12345,"[{'descripcio': 'CROQUETES', 'import': '1,05',...","1,38",24052024_184507,2,24052024
4,SANTA COLOMA DE FARNERS 17430,22-04-2024,1234-123-12345,"[{'descripcio': 'IOGURT NATURAL', 'import': '3...","58,90",24052024_184201,3,24052024
5,BARCELONA 08019,22-04-2024,1234-123-12345,"[{'descripcio': 'CALAMARSO.', 'import': '16,50...","19,83",24052024_184226,3,24052024
6,VILANOVA I LA GELTRÚ 08800,22-04-2024,1234-123-12345,"[{'descripcio': 'CANELONS', 'import': '4.62', ...","23,95",24052024_184128,4,24052024
7,MONTORNÉS DEL VALLES 08170,22-04-2024,1234-123-12345,"[{'descripcio': 'PATATES GRILL', 'import': '1,...","3,50",24052024_184542,4,24052024


In [0]:
# Natajem el camp ciutat treien el codi postal
df_docs['ciutat'] = df_docs['ciutat'].apply(lambda x: x.split(' ')[0])
df_docs

,ciutat,data,factura_simplificada,products,total_amb_IVA,id_tiquet,id_client,data_scanner_tiquet
0,VILADECANS,22-04-2024,1234-123-12345,"[{'descripcio': 'LASANYA', 'import': '17,60', ...","17,60",24052024_184100,1,24052024
1,REUS,22-04-2024,1234-123-12345,"[{'descripcio': 'ARROS NEGRE', 'import': '2.3)...","30,8]",24052024_184112,2,24052024
2,BARCELONA,22-04-2024,1234-123-12345,"[{'descripcio': 'ANELLES POTA', 'import': '4.3...","42,00",24052024_184138,2,24052024
3,BARCELONA,22-04-2024,1234-123-12345,"[{'descripcio': 'CROQUETES', 'import': '1,05',...","1,38",24052024_184507,2,24052024
4,SANTA,22-04-2024,1234-123-12345,"[{'descripcio': 'IOGURT NATURAL', 'import': '3...","58,90",24052024_184201,3,24052024
5,BARCELONA,22-04-2024,1234-123-12345,"[{'descripcio': 'CALAMARSO.', 'import': '16,50...","19,83",24052024_184226,3,24052024
6,VILANOVA,22-04-2024,1234-123-12345,"[{'descripcio': 'CANELONS', 'import': '4.62', ...","23,95",24052024_184128,4,24052024
7,MONTORNÉS,22-04-2024,1234-123-12345,"[{'descripcio': 'PATATES GRILL', 'import': '1,...","3,50",24052024_184542,4,24052024


In [0]:
#  Convertim la columna 'data_scanner_tiquet' a datetime.
df_docs['data_scanner_tiquet'] = pd.to_datetime(df_docs['data_scanner_tiquet'], format='%d%m%Y')

# Ara utilitzem dt.strftime per canviar-li el format.
df_docs['data_scanner_tiquet'] = df_docs['data_scanner_tiquet'].dt.strftime('%d-%m-%Y')

In [0]:
# Creem la nova columna "unitat".
df_docs = crear_columna_unitats(df_docs)
# Comprovem si ha funcionat.
visualitzarProducts(df_docs)

Tiquet  1

--Productes--
Nom: LASANYA
Qty: 4
Import: 17,60 €
preu_unitari: 4.40 €
unitat: ud

Tiquet  2

--Productes--
Nom: ARROS NEGRE
Qty: 1
Import: 2.3) €
preu_unitari: 2.3) €
unitat: ud

Nom: GELAT TORRO
Qty: 1
Import: 2.75 €
preu_unitari: 2.75 €
unitat: ud

Nom: ARROS PRECUIT
Qty: 1
Import: 1,72 €
preu_unitari: 1,72 €
unitat: ud

Nom: LLOM HACENDADO
Qty: 1
Import: 6.05 €
preu_unitari: 6.05 €
unitat: ud

Nom: GELAT AMETLLAT
Qty: 3
Import: 6.93 €
preu_unitari: 231 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 1
Import: 11,06 €
preu_unitari: 11,06 €
unitat: ud

Tiquet  3

--Productes--
Nom: ANELLES POTA
Qty: 1
Import: 4.35 €
preu_unitari: 4.35 €
unitat: ud

Nom: PIZZA CAMPEROLA
Qty: 2
Import: 440 €
preu_unitari: 2,20 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 4
Import: 30.80 €
preu_unitari: 7,70 €
unitat: ud

Nom: BLAT MORO DOLG
Qty: 1
Import: 1,26 €
preu_unitari: 1,26 €
unitat: ud

Nom: LLEIXIU AMB
Qty: 1
Import: 1,19 €
preu_unitari: 1,19 €
unitat: ud

Tiquet  4

--Productes--
Nom: CROQUETES

In [0]:
# Canviem els "." per "," a variables de productes.
df_docs = df_docs.apply(canviar_punts_per_comes, axis=1)
visualitzarProducts(df_docs)

Tiquet  1

--Productes--
Nom: LASANYA
Qty: 4
Import: 17.60 €
preu_unitari: 4.40 €
unitat: ud

Tiquet  2

--Productes--
Nom: ARROS NEGRE
Qty: 1
Import: 2.3) €
preu_unitari: 2.3) €
unitat: ud

Nom: GELAT TORRO
Qty: 1
Import: 2.75 €
preu_unitari: 2.75 €
unitat: ud

Nom: ARROS PRECUIT
Qty: 1
Import: 1.72 €
preu_unitari: 1.72 €
unitat: ud

Nom: LLOM HACENDADO
Qty: 1
Import: 6.05 €
preu_unitari: 6.05 €
unitat: ud

Nom: GELAT AMETLLAT
Qty: 3
Import: 6.93 €
preu_unitari: 231 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 1
Import: 11.06 €
preu_unitari: 11.06 €
unitat: ud

Tiquet  3

--Productes--
Nom: ANELLES POTA
Qty: 1
Import: 4.35 €
preu_unitari: 4.35 €
unitat: ud

Nom: PIZZA CAMPEROLA
Qty: 2
Import: 440 €
preu_unitari: 2.20 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 4
Import: 30.80 €
preu_unitari: 7.70 €
unitat: ud

Nom: BLAT MORO DOLG
Qty: 1
Import: 1.26 €
preu_unitari: 1.26 €
unitat: ud

Nom: LLEIXIU AMB
Qty: 1
Import: 1.19 €
preu_unitari: 1.19 €
unitat: ud

Tiquet  4

--Productes--
Nom: CROQUETES

In [0]:
# Treiem el text kg de la columna preu_unitari
df_docs = eliminar_txt_kg(df_docs)
visualitzarProducts(df_docs)

Tiquet  1

--Productes--
Nom: LASANYA
Qty: 4
Import: 17.60 €
preu_unitari: 4.40 €
unitat: ud

Tiquet  2

--Productes--
Nom: ARROS NEGRE
Qty: 1
Import: 2.3) €
preu_unitari: 2.3) €
unitat: ud

Nom: GELAT TORRO
Qty: 1
Import: 2.75 €
preu_unitari: 2.75 €
unitat: ud

Nom: ARROS PRECUIT
Qty: 1
Import: 1.72 €
preu_unitari: 1.72 €
unitat: ud

Nom: LLOM HACENDADO
Qty: 1
Import: 6.05 €
preu_unitari: 6.05 €
unitat: ud

Nom: GELAT AMETLLAT
Qty: 3
Import: 6.93 €
preu_unitari: 231 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 1
Import: 11.06 €
preu_unitari: 11.06 €
unitat: ud

Tiquet  3

--Productes--
Nom: ANELLES POTA
Qty: 1
Import: 4.35 €
preu_unitari: 4.35 €
unitat: ud

Nom: PIZZA CAMPEROLA
Qty: 2
Import: 440 €
preu_unitari: 2.20 €
unitat: ud

Nom: PASTIS INFANTIL
Qty: 4
Import: 30.80 €
preu_unitari: 7.70 €
unitat: ud

Nom: BLAT MORO DOLG
Qty: 1
Import: 1.26 €
preu_unitari: 1.26 €
unitat: ud

Nom: LLEIXIU AMB
Qty: 1
Import: 1.19 €
preu_unitari: 1.19 €
unitat: ud

Tiquet  4

--Productes--
Nom: CROQUETES


#### Emmagatzemem les dades a la capa Silver.
Per emmagatzemar les dades a la capa Silver, utilitzarem el format Parquet. Aquesta decisió té diverses avantatges:

- Reducció de tamany de les dades:
  - El format Parquet és altament eficient en la compressió de dades, cosa que ens permet reduir el tamany que ocupen les dades emmagatzemades.

- Particionament eficient:
  - Podem emmagatzemar les dades de manera particionada, la qual cosa fa que sigui molt més fàcil i eficient accedir a subconjunts específics de les dades.
  - Particionarem les dades tant per clients com per dies, ja que això ens permetrà organitzar-les i accedir-hi de manera intuïtiva i eficient.

Per particionar les dades per clients i dies, crearem dues carpetes dins de la carpeta Silver:

1- Particionament per clients:
  - Cada subdirectori dins de la carpeta Silver correspon al ID del client.
  - Aquesta partició ens permet agrupar totes les dades relacionades amb un mateix client en una única ubicació, facilitant-ne l'accés i l'anàlisi.

2- Particionament per dies:
  - Cada subdirectori dins de la carpeta Silver té el format de la data del tiquet (dd/mm/YYYY).
  - Aquesta partició ens permet agrupar les dades segons la data del tiquet, el que facilita l'anàlisi temporal i l'accés a les dades en funció de la seva data d'origen.

Utilitzarem aquest enfocament per emmagatzemar les dades a la capa Silver de manera ordenada i eficient, assegurant-nos que estiguin disponibles i fàcilment accessibles per a futures anàlisis i ús.


In [0]:
# Penjem les dades particionades amb dies i clients.

# DIES (DATA)
path_dies_silver = 'abfs://{0}@{1}.dfs.core.windows.net/dies/'.format('silver', storage_account)

# CLIENTS
path_clientes_silver = 'abfs://{0}@{1}.dfs.core.windows.net/clients/'.format('silver', storage_account)

# Executem la funció per dies
particionar_dies(df_docs, path_dies_silver)

# Executem la funció per clients
particionar_clients(df_docs, path_clientes_silver)